In [2]:
## all imports
import pandas as pd
from IPython.display import HTML
import numpy as np
import bs4 #this is beautiful soup
import time
import operator
import socket
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

"""import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
"""""

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

print("Hello")

Hello


In [4]:
# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']

# read csv file which onlinne by writing pd.read.csv('url', sep='|', names)
users = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.user', 
    sep='|', names=u_cols)

#only call first 5 row
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [41]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 
    sep='\t', names=r_cols)

ratings.head() 

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [10]:
# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 
            'video_release_date', 'imdb_url']

movies = pd.read_csv(
    'http://files.grouplens.org/datasets/movielens/ml-100k/u.item', 
    sep='|', names=m_cols, usecols=range(5), encoding='latin-1')

movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


# Get information about data

In [14]:
print (movies.dtypes)
print (movies.describe())

movie_id                int64
title                  object
release_date           object
video_release_date    float64
imdb_url               object
dtype: object
          movie_id  video_release_date
count  1682.000000                 0.0
mean    841.500000                 NaN
std     485.695893                 NaN
min       1.000000                 NaN
25%     421.250000                 NaN
50%     841.500000                 NaN
75%    1261.750000                 NaN
max    1682.000000                 NaN


In [17]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [18]:
users['occupation'].head()
## *** Where did the nice design go? ***

0    technician
1         other
2        writer
3    technician
4         other
Name: occupation, dtype: object

In [21]:
columns_you_want = ['occupation', 'sex'] 
users[columns_you_want].head()

,occupation,sex
0,technician,M
1,other,F
2,writer,M
3,technician,M
4,other,F


In [23]:
print (users.iloc[3])


user_id                4
age                   24
sex                    M
occupation    technician
zip_code           43537
Name: 3, dtype: object


In [24]:
print (users.head())


   user_id  age sex  occupation zip_code
0        1   24   M  technician    85711
1        2   53   F       other    94043
2        3   23   M      writer    32067
3        4   24   M  technician    43537
4        5   33   F       other    15213


# Filtering data

Select users older than 25


In [27]:
OldUser = users[users.age>25]
OldUser.head()

,user_id,age,sex,occupation,zip_code
1,2,53,F,other,94043
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201


# Quiz:

show users aged 40 and male

show the mean age of female programmers

In [31]:
User40Male = users[(users.age == 40) & (users.sex =='M')]
User40Male.head()

,user_id,age,sex,occupation,zip_code
18,19,40,M,librarian,02138
82,83,40,M,other,44133
115,116,40,M,healthcare,97232
199,200,40,M,programmer,93402
283,284,40,M,executive,92629


In [38]:
MeanFP = users[(users.sex =='F') & (users.occupation == 'programmer')]
## show statistic summary
print (MeanFP.describe())

MeanFP.head()


          user_id        age
count    6.000000   6.000000
mean   411.166667  32.166667
std    149.987222   5.115336
min    292.000000  26.000000
25%    313.000000  28.250000
50%    378.000000  32.000000
75%    416.750000  36.500000
max    698.000000  38.000000


,user_id,age,sex,occupation,zip_code
291,292,35,F,programmer,94703
299,300,26,F,programmer,55106
351,352,37,F,programmer,55105
403,404,29,F,programmer,55108
420,421,38,F,programmer,55105


In [39]:
## alternatives:
print (MeanFP.age.mean())
print (MeanFP['age'].mean())

32.166666666666664
32.166666666666664


# Split-apply-combine

-splitting the data into groups based on some criteria

-applying a function to each group independently

-combining the results into a data structure

In [60]:
print (ratings.head())

   user_id  movie_id  rating  unix_timestamp
0      196       242       3       881250949
1      186       302       3       891717742
2       22       377       1       878887116
3      244        51       2       880606923
4      166       346       1       886397596


In [72]:
#print (ratings.head())
## split data
grouped_data = ratings.groupby('user_id')
grouped_data = ratings['movie_id'].groupby(ratings['user_id'])

## count and combine
ratings_per_user = grouped_data.count()
ratings_per_user.head(5)


user_id
1    272
2     62
3     54
4     24
5    175
Name: movie_id, dtype: int64

# Quiz

-get the average rating per movie

-advanced: get the movie titles with the highest average rating

In [68]:
## split data
grouped_data = ratings['rating'].groupby(ratings['movie_id'])

## average and combine
average_ratings = grouped_data.mean()
print ("Average ratings:")
print (average_ratings.head())

#Max and Min of rating
maximum_rating = average_ratings.max()
print ('Max-rating:', maximum_rating)
minimum_rating = average_ratings.min()
print ('Min-rating:',minimum_rating)

Average ratings:
movie_id
1    3.878319
2    3.206107
3    3.033333
4    3.550239
5    3.302326
Name: rating, dtype: float64
Max-rating: 5.0
Min-rating: 1.0


In [76]:
good_movie_ids = average_ratings[average_ratings == maximum_rating].index
print ("Good movie ids:")
print (good_movie_ids)

print ("Best movie titles")
print (movies[movies.movie_id.isin(good_movie_ids)].title)

Good movie ids:
Int64Index([814, 1122, 1189, 1201, 1293, 1467, 1500, 1536, 1599, 1653], dtype='int64', name='movie_id')
Best movie titles
813                         Great Day in Harlem, A (1994)
1121                       They Made Me a Criminal (1939)
1188                                   Prefontaine (1997)
1200           Marlene Dietrich: Shadow and Light (1996) 
1292                                      Star Kid (1997)
1466                 Saint of Fort Washington, The (1993)
1499                            Santa with Muscles (1996)
1535                                 Aiqing wansui (1994)
1598                        Someone Else's America (1995)
1652    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object


In [79]:
how_many_ratings = grouped_data.count()
print ("Number of ratings per movie")
print (how_many_ratings[average_ratings == maximum_rating])

Number of ratings per movie
user_id
814    35
Name: movie_id, dtype: int64
